In [26]:
import pandas as pd
from fpdf import FPDF, HTMLMixin

dept = 'DEPARTMENT OF COMPUTER ENGINEERING'
df = pd.read_excel('TotalMarks.xlsx').iloc[:,1:]

def toppertable(top3):
    html = """
    <font size=10>
    <table border="0" align="center" width="80%">
    <thead>
    <tr>
    <th width="15%">ID</th>
    <th width="70%">Name</th>
    <th width="15%">SGPI</th>
    </tr>
    </thead>
    <tbody >
    """
    s = """"""
    for i in range(len(top3)):
        s = s + '''
        <tr>
        <td align="center" width="15%">'''+str(i+1)+'''</td>
        <td align="center" width="70%">'''+str(top3.iloc[i,1]).strip('/')+'''</td>
        <td align="center" width="15%">'''+str(top3.iloc[i,-2])+'''</td>
        </tr>'''
    html = html + s
    s = """
    </tbody>
    </table>
    </font>
    """
    html = html + s
    return html

def subtoppertable(subtopper):
    html = ''''''
    for k,v in subtopper.items():
        html = html + """<br>
        <font size=10>
        <b align="Left">""" + k.split("-")[0] + """<b>
        <table border="0" align="center" width="80%">
        <thead>
        <tr>
        <th width="15%">Subject</th>
        <th width="70%">Name</th>
        <th width="15%">Marks</th>
        </tr>
        </thead>
        <tbody >"""
        s = ''
        for i in v:
            s = s + '''
            <tr>
            <td align="center" width="15%">'''+str(i[0]+1)+'''</td>
            <td align="center" width="70%">'''+str(i[1]).strip('/')+'''</td>
            <td align="center" width="15%">'''+str(i[2])+'''</td>
            </tr>
            '''
        html = html + s +"""
        </tbody>
        </table>
        </font>
        """
    return html

def overalltable(overallresult):
    html = '''
    <font size=10>
    <table border="0" align="center" width="100%">
    <thead>
    <tr>
    <th width="52%">Subject</th>
    <th width="8%">Total</th>
    <th width="8%">Pass</th>
    <th width="8%">Pass%</th>
    <th width="8%">40-50</th>
    <th width="8%">50-60</th>
    <th width="8%">60+</th>
    </tr>
    </thead>
    <tbody >'''
    s = ''
    for i in range(len(overallresult)):
        s = s + '''
        <tr>
        <td align="center" width="52%">'''+str(overallresult.iloc[i,0])+'''</td>
        <td align="center" width="8%">'''+str(overallresult.iloc[i,1])+'''</td>
        <td align="center" width="8%">'''+str(overallresult.iloc[i,2])+'''</td>
        <td align="center" width="8%">'''+str(overallresult.iloc[i,3])+'''</td>
        <td align="center" width="8%">'''+str(overallresult.iloc[i,4])+'''</td>
        <td align="center" width="8%">'''+str(overallresult.iloc[i,5])+'''</td>
        <td align="center" width="8%">'''+str(overallresult.iloc[i,6])+'''</td>
        </tr>'''
    
    html = html + s + '''
    </tbody>
    </table>
    </font>
    '''
    return html



In [27]:
class PDF(FPDF, HTMLMixin):
    def header(self):
        self.image('temp/RGIT.png', x = 25, h=30)

# pdf.cell(w, h = 0, txt = '', border = 0, ln = 0, align = '', fill = False, link = '')

pdf = PDF()
pdf.add_page()
pdf.dashed_line(10, 35, 200, 35)
pdf.set_font('Arial', 'B', 14)
pdf.cell(0,5,dept, 0, 1 ,'C')

# TOP3
pdf.set_font('Arial', 'B', 12)
pdf.cell(0,5,'Overall Topper:', 0, 1 ,'L')
pdf.cell(0, ln=1)

top3 = df.sort_values(['SGPI','TOTAL'], axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last').iloc[:3,:]
pdf.set_font('Arial', '', 12)
pdf.write_html(toppertable(top3))

# SUBJECTTOPPERS
pdf.cell(0, ln=1)
pdf.set_font('Arial', 'B', 12)
pdf.cell(0,5,'Subject Toppers:', 0, 1 ,'L')


subtoppers = {}

for i in df.columns[2:-3]:
    sub2 = df.sort_values(i, axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last').iloc[:2,:]
    for j in range(2):
        if j == 0:
            subtoppers[i] = [0,sub2.iloc[j,1], sub2.iloc[j,list(df.columns).index(i)]]
        else:
            subtoppers[i] = [subtoppers[i],[1,sub2.iloc[j,1], sub2.iloc[j,list(df.columns).index(i)]]]
            
pdf.cell(0, ln=1)
pdf.set_font('Arial', 'B', 14)
pdf.write_html(subtoppertable(subtoppers))

In [28]:
overallresult = pd.DataFrame(columns = ['Subjects', 'Appeared', 'Passed','Pass%','40-50','51-59','60+']) 
overallresult['Subjects'] = df.columns[2:-3]
app = []
passed = []
psp = []
m1 = []
m2 = []
m3 = []
for i in df.columns[2:-3]:
    app.append(sum(df[i].apply(lambda x: x>0)))
    passed.append(sum(df[i].apply(lambda x: x>40)))
    psp.append(round(passed[-1]*100/app[-1],2) )
    m1.append(sum(df[i].apply(lambda x: x>=40 and x < 50)))
    m2.append(sum(df[i].apply(lambda x: x>=50 and x < 60)))
    m3.append(sum(df[i].apply(lambda x: x>=60)))

overallresult['Appeared'] = app
overallresult['Passed'] = passed
overallresult['Pass%'] = psp
overallresult['40-50'] = m1
overallresult['51-59'] = m2
overallresult['60+'] = m3
overallresult['Subjects'] = overallresult['Subjects'].apply(lambda x: x.split("-")[0])

pdf.cell(0, ln=1)
pdf.set_font('Arial', 'B', 12)
pdf.cell(0,5,'Result Summary:', 0, 1 ,'L')


pdf.set_font('Arial', 'B', 11)
pdf.cell(0,5,'Result :- '+str(round(sum(df['RESULT'].apply(lambda x : x == 'P'))*100/len(df),2)), 0, 1 ,'L')


pdf.write_html(overalltable(overallresult))

pdf.cell(0, ln=1)
pdf.set_font('Arial', 'B', 12)
pdf.cell(0,5,'Statistical Analysis:', 0, 1 ,'L')
pdf.image('SUB.jpg', x = 0, h = 100)

pdf.cell(0, ln=1)
pdf.set_font('Arial', 'B', 12)
pdf.cell(0,5,'SGPI Statistical Analysis:', 0, 1 ,'L')
pdf.image('SGPI.jpg', x = 0, h = 100)


pdf.output('ResultAnalysis.pdf', 'F')

''